In [1]:
from os import listdir
from os.path import isfile, join
import os
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import time
import numpy as np
import pickle
import json
from tqdm.notebook import tqdm
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
# Парсинг условий продажи

# url = 'https://www.tomsk.ru09.ru/realty?subaction=detail&id=4414402'
# test_soup = get_soup_by_url(url)

# for i in df.index[0:10]:
#     test_soup = get_soup_by_url(i)
#     if
#     print(test_soup.find('sup').parent.text)

# get_soup_by_url(df.index[0])
# requests.get(df.index[0])

In [2]:
pd.set_option('display.max_rows', 2000)
sns.set(rc={'figure.figsize': (16, 12)})
%matplotlib inline

In [3]:
def get_soup_by_url(url):
    
    html_ = requests.get(url).text
    soup = BeautifulSoup(html_, 'lxml')
    
    return soup    

In [4]:
# Получаем номер последней страницы
def get_number_last_page():
    
    soup = get_soup_by_url('https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page=1')
    number_last_page = int(soup.find('td', {'class':'pager_pages'}).find_all('a')[4].text)
    
    return number_last_page

In [5]:
def find_district_field(keys):
    
    for i, j in enumerate(keys):
        if 'район' in j:
            break
    return i

In [6]:
def parse_apartment(url):
#     headers = {'User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
#     start_time = time.time()
    
    soup = get_soup_by_url(url)
    
    keys = [i.find('span').text.replace('\xa0','').lower() for i in soup.find_all('tr', {'class': 'realty_detail_attr'})]
    
    district_idx = find_district_field(keys)
    items = {'район': keys[district_idx]}
    
    keys = [j for i, j in enumerate(keys) if i not in (district_idx - 1, district_idx)]
    values = [i.text.replace('\xa0', ' ') for i in soup.find_all(class_='nowrap')]
    
    items.update(dict(zip(keys, values)))
    
    items['адрес'] = soup.find(class_='table_map_link').text.replace('\xa0', ' ')
    items['цена'] = int(soup.find('div', {'class': 'realty_detail_price inline'}).text.replace('\xa0','').replace('руб.',''))
    items['ид'] = int(soup.find('strong').text)
    items['дата добавления'] = soup.find(class_='realty_detail_date nobr').get('title')
    items['дата истечения'] = soup.find_all(class_='realty_detail_date')[4].get('title')
    
    return items

In [16]:
def handle_dataframe(df):

    
    def extract_number_from_squares(x):
    
        if isinstance(x, float):
            value = x
        else:
            value = float(x.split(' ')[0])
    
        return value

    
    columns_order = [#'ид', 
                     'район', 'адрес', 
                     'вид',
                     'год постройки', 'материал', 
                     'этажность', 'этаж', 'тип квартиры', 'цена', 'общая площадь', 
                     'жилая', 'кухня', 'количество комнат', 'отделка', 'санузел', 
                     'балкон/лоджия', 'дата добавления', 'дата истечения']
    
    rename_columns = {#'ид': 'ID', 
                      'район': 'District', 
                      'адрес': 'Address', 
                      'вид': 'Sales_Type', 
                      'год постройки': 'Year_Building', 
                      'материал': 'Material', 
                      'этажность': 'Floors_In_Building', 
                      'этаж': 'Floor', 
                      'тип квартиры': 'Apartment_Type', 
                      'цена': 'Price', 
                      'общая площадь': 'Square_Total', 
                      'жилая': 'Square_Living', 
                      'кухня': 'Square_Kitchen', 
                      'количество комнат': 'Rooms_Number', 
                      'отделка': 'Apartment_Condition', 
                      'санузел': 'Bathroom_Type', 
                      'балкон/лоджия': 'Balcony_Loggia', 
                      'дата добавления': 'Date_Add', 
                      'дата истечения': 'Date_Expiration'}
    
    df[['общая площадь','кухня','жилая']] = df[['общая площадь','кухня','жилая']].applymap(extract_number_from_squares)    
        
    df['дата добавления'] = pd.to_datetime(df['дата добавления'], format='%d.%m.%Y %H:%M:%S')
    df['дата истечения'] = pd.to_datetime(df['дата истечения'], format='%d.%m.%Y')
    df['этаж'] = [int(i[0]) if i[0].isdigit() else 0 for i in df['этаж/этажность'].str.split('/')]
    df['цена'] = df['цена'] / 1000
    
    df.drop(['этаж/этажность', 'ид'], axis=1, inplace=True)
    
    df = df[columns_order]
    df = df.rename(columns = rename_columns)
    
    return df

In [8]:
def get_urls_pages(start_page=1, end_page=None):
    
    url_base = 'https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page='
    
    end_page = end_page or get_number_last_page()
    pages_to_parse = range(start_page, end_page + 1)
    urls_pages = [url_base + str(i) for i in pages_to_parse]
        
    return urls_pages

In [9]:
def get_urls_apartments_by_page(url_page):
    
    url_base = 'https://www.tomsk.ru09.ru'
    
    soup = get_soup_by_url(url_page)
    soup = soup.find_all('a', {'class':'visited_ads'})

    urls_apartments = set()
    
    for i in soup:
        urls_apartments.add(url_base + i.get('href'))
    
    return urls_apartments

In [10]:
def main(start_page=1, end_page=None, filename='data.json'):
    
    urls_pages = get_urls_pages(start_page, end_page)
    path = 'C:/Users/qwerty.Oleg/'
    
    if filename in listdir(path):
        with open(filename, 'r') as fp:
            storage_dict = json.load(fp)
    else:
        storage_dict = {}
    
    len_storage = len(storage_dict)
    print('Apartments in storage: {}\n'.format(len_storage))
    
    for url_page in tqdm(urls_pages, desc='Pages'):
        urls_apartments = get_urls_apartments_by_page(url_page)
        urls_apartments_to_parse = urls_apartments.difference(set(storage_dict))
        
        if len(urls_apartments_to_parse) != 0:
            for url_apartment in tqdm(urls_apartments_to_parse, desc='Apartments', leave=False):
                storage_dict[url_apartment] = parse_apartment(url_apartment)
        
    with open(filename, 'w') as fp:
        json.dump(storage_dict, fp)
            
    print('New apartments: {}'.format(len(storage_dict)-len_storage))

In [11]:
main()

Apartments in storage: 10324




New apartments: 47


In [ ]:
pd.read_json('data.json', orient='index')

In [12]:
df = pd.read_json('data.json', orient='index')
df = handle_dataframe(df)
df

,ID,District,Address,Sales_Type,Year_Building,Material,Floors_In_Building,Floor,Apartment_Type,Price,Square_Total,Square_Living,Square_Kitchen,Rooms_Number,Apartment_Condition,Bathroom_Type,Balcony_Loggia,Date_Add,Date_Expiration
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4353796,4353796.0,советский район,Фрунзе проспект 126,вторичное,1975.0,кирпич,5,3,гостинка,1250.0,18.0,NaN,NaN,1,в хорошем состоянии,совмещенный,NaN,2020-03-15 10:29:07,2020-04-12
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4207812,4207812.0,советский район,Вершинина 7,вторичное,1996.0,кирпич,5,2,NaN,13900.0,200.0,NaN,NaN,4,в отличном состоянии,раздельный,"лоджия, остекление",2020-03-12 22:44:17,2020-06-10
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4398582,4398582.0,советский район,Источная 10,вторичное,2009.0,монолит,7,5,NaN,6900.0,81.4,NaN,NaN,3,в отличном состоянии,раздельный,лоджия,2020-03-15 10:29:09,2020-04-12
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4119652,4119652.0,советский район,Енисейская 4,вторичное,1987.0,кирпич,9,5,секционка,1750.0,33.0,NaN,NaN,2,в отличном состоянии,раздельный,NaN,2020-03-15 10:25:13,2020-05-27
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4368552,4368552.0,ленинский район,Заозерный переулок 16/2,вторичное,1972.0,кирпич,5,3,гостинка,1100.0,17.2,13.0,NaN,1,NaN,NaN,NaN,2020-03-15 10:07:04,2020-04-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4416644,4416644.0,ленинский район,Розы Люксембург 135,вторичное,NaN,NaN,3,3,гостинка,1620.0,22.0,NaN,NaN,1,в отличном состоянии,совмещенный,"лоджия, остекление",2020-03-30 22:34:08,2020-05-14
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4416642,4416642.0,кировский район,Ботанический переулок 10,вторичное,1991.0,панель,5,5,NaN,1750.0,44.7,30.0,6.0,2,в хорошем состоянии,раздельный,"балкон, остекление",2020-03-30 22:32:07,2020-05-14
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4416632,4416632.0,кировский район,19-й Гвардейской Дивизии 15,вторичное,1983.0,панель,9,9,NaN,2250.0,36.0,NaN,NaN,1,в хорошем состоянии,совмещенный,балкон,2020-03-30 22:28:17,2020-05-14
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4416634,4416634.0,советский район,Фрунзе проспект 216,вторичное,1962.0,кирпич,5,3,NaN,1700.0,29.0,NaN,NaN,1,в хорошем состоянии,совмещенный,NaN,2020-03-30 22:29:03,2020-05-14
